In [2]:
pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 495.5 kB/s eta 0:05:03
   ---------------------------------------- 0.1/150.0 MB 880.9 kB/s eta 0:02:51
   ---------------------------------------- 0.1/150.0 MB 880.9 kB/s eta 0:02:51
   ---------------------------------------- 0.2/150.0 MB 908.0 kB/s eta 0:02:45
   ---------------------------------------- 0.4/150.0 MB 1.4 MB/s eta 0:01:48
   ---------------------------------------- 0.5/150.0 MB 1.7 MB/s eta 0:01:27
   ---------------------------------------- 0.7/150.0 MB 2.0 MB/s eta 0:01:15
   ---------------------------------------- 0.8/150.0 MB 2.2 MB/s eta 0:01:07
   ---------------------------------------- 1.0/150.0 MB 2.4 MB/s eta 0:01:03
   ---------------------------------------- 1.1/150.0 MB 2.3 MB/s eta 0:01:06
   ---------------------------------------- 1.3/150.0 MB 2.5 MB/s eta 

In [1]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

file_path = "Adjusted_Dataset.csv"
df = pd.read_csv(file_path)

target_column = "Risk Score (%)"  
X = df.drop(columns=[target_column,"Risk Category","Predicted Risk Category","Risk Category Encoded"])
y = df[target_column]

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)


joblib.dump(model, "risk_score_xgboost_model.pkl")
joblib.dump(scaler, "scaler.pkl")


Mean Absolute Error (MAE): 3.9809741973876953
Mean Squared Error (MSE): 27.446083068847656
R² Score: 0.9787577390670776


['scaler.pkl']

In [ ]:
model = joblib.load("risk_score_xgboost_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoders = joblib.load("label_encoders.pkl")  

FEATURES = ["State", "City", "Crop", "pH soil", "Area (ha)", "avgTemp", 
            "Rainfall (mm)", "Yield (ton/ha)", "Avg_smlvl", "Soil Type", 
            "Nutrient Level", "Credit Score"]

app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()

        input_df = pd.DataFrame([data])

        input_df = input_df.reindex(columns=FEATURES, fill_value=0)

        categorical_columns = ["State", "City", "Crop", "Soil Type", "Nutrient Level"]
        for col in categorical_columns:
            if col in input_df.columns:
                if col in label_encoders: 
                    input_df[col] = input_df[col].apply(lambda x: label_encoders[col].transform([x])[0] 
                                                         if x in label_encoders[col].classes_ else -1)

        scaled_data = scaler.transform(input_df)

        prediction = model.predict(scaled_data)[0]

        return jsonify({"Predicted Risk Score (%)": float(prediction)})
    
    except Exception as e:
        return jsonify({"error": str(e)})
        
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8080, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.52.1:8080
Press CTRL+C to quit
127.0.0.1 - - [23/Mar/2025 12:04:53] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 12:24:50] "POST /predict HTTP/1.1" 200 -
